In [2]:
import os
import shutil

# Define paths
original_segmented_dir = 'Datasets'  
final_dataset_dir = 'final_dataset'     

# Ensure the final dataset directory exists
os.makedirs(final_dataset_dir, exist_ok=True)

# Process each class folder (Bearing_fault, Normal_state, Unbalance_weight)
for class_name in os.listdir(original_segmented_dir):
    class_path = os.path.join(original_segmented_dir, class_name)
    
    # Skip non-directory files (if any)
    if not os.path.isdir(class_path):
        continue
    
    # Process each segmented duration folder within the class folder
    for segmented_folder in os.listdir(class_path):
        segmented_folder_path = os.path.join(class_path, segmented_folder)
        
        # Skip non-directory files
        if not os.path.isdir(segmented_folder_path):
            continue
        
        # Extract duration and overlap info from the folder name (e.g., "segmented_5" -> 5s, "segmented_10_overlap" -> 10s_overlap)
        folder_parts = segmented_folder.split('_')
        duration = folder_parts[1]  # Get the duration part (5, 10, 15)
        
        # Check if the folder is for overlapping segments
        if 'overlap' in folder_parts:
            new_duration_folder = f"{duration}s_overlap"
        else:
            new_duration_folder = f"{duration}s"
        
        # Create the target directory for this duration and class
        target_dir = os.path.join(final_dataset_dir, new_duration_folder, class_name)
        os.makedirs(target_dir, exist_ok=True)
        
        # Copy all video segments from the source to the target directory
        for segment_file in os.listdir(segmented_folder_path):
            src_path = os.path.join(segmented_folder_path, segment_file)
            dest_path = os.path.join(target_dir, segment_file)
            
            # Copy the file
            shutil.copy2(src_path, dest_path)
            print(f"Copied: {src_path} -> {dest_path}")

print("Dataset reorganization completed!")

Copied: Datasets\Bearing_fault\segmented_10\segmented_10_features.csv -> final_dataset\10s\Bearing_fault\segmented_10_features.csv
Copied: Datasets\Bearing_fault\segmented_10_overlap\segmented_10_overlap_features.csv -> final_dataset\10s_overlap\Bearing_fault\segmented_10_overlap_features.csv
Copied: Datasets\Bearing_fault\segmented_15\segmented_15_features.csv -> final_dataset\15s\Bearing_fault\segmented_15_features.csv
Copied: Datasets\Bearing_fault\segmented_15_overlap\segmented_15_overlap_features.csv -> final_dataset\15s_overlap\Bearing_fault\segmented_15_overlap_features.csv
Copied: Datasets\Bearing_fault\segmented_5\segmented_5_features.csv -> final_dataset\5s\Bearing_fault\segmented_5_features.csv
Copied: Datasets\Bearing_fault\segmented_5_overlap\segmented_5_overlap_features.csv -> final_dataset\5s_overlap\Bearing_fault\segmented_5_overlap_features.csv
Copied: Datasets\Normal_state\segmented_10\segmented_10_features.csv -> final_dataset\10s\Normal_state\segmented_10_features.c